### Project checklist

- [ ] Title
- [ ] Abstract (max 300 words)
- [ ] env.yml (include both full and cross-platform)
  - _If time..._
    - [ ] Set up container environment to run Notebook (Binder?)
- [ ] Package motivations
- [ ] Include rich text (equations/tables/links/images/vids)
- **I/O**
  - [ ] Use `pandas` to read large data _or_ `numpy` to load from files
  - [ ] Save processed/generated data to disk with `pandas`
- **DATA MANIPULATION**
  - [ ] Needs to include numerical operations (`numpy`, `scipy`, `pandas`) or data transformation (`pandas`)
- **VISUALIZATION**
  - [ ] Min. one composite plot (multi-panel or inset)
  - "[Publication ready figures](https://pubs.acs.org/doi/10.1021/jz500997e)"
    - [ ] The figs are 89 mm wide (single column) or 183 mm wide (double column)
    - [ ] The axes are labeled
    - [ ] The font sizes are sufficiently large
    - [ ] The figures are saved as ~~rasterized images (300 dpi) or~~ **vector art**
- [ ] [Repo Zenodo DOI](https://docs.github.com/en/repositories/archiving-a-github-repository/referencing-and-citing-content)

# Project title

## Abstract

Abstract text...

## Index

- [Notebook instructions](#notebook-instructions)
- [Packages](#packages)
  - [Package](#package-name)

## Notebook instructions

_Information on how to use/run the notebook_.

## Packages

### Package name

_Reason for inclusion_.